In [ ]:
import xgboost as xgb
import pandas as pd


from sklearn.model_selection import GridSearchCV


In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
clf = xgb.XGBClassifier(tree_method="hist", random_state =123)


In [ ]:
all_columns_fit = [col for col in train.columns if col not in ['step', 'isFraud', 'isFlaggedFraud', 'nameOrig', 'nameDest' ]]
all_columns_fit